In [1]:
##
# Lets create the file
##

import turicreate as tc
import csv

with open('trainingData.csv', 'rb') as trainingData:
    sf = tc.SFrame.read_csv('trainingData.csv', delimiter=',', header=True, verbose=False)
        
sf.save('trainingData.sframe')

with open('validationData.csv', 'rb') as validationData :
    vsf = tc.SFrame.read_csv('validationData.csv', delimiter=',', header=True, verbose=False)
    
vsf.save('validationData.sframe')


In [2]:
##
# Let manipulate said file - Oh yea
##

import turicreate as tc

# Load sessions from preprocessed data
data = tc.SFrame('validationData.sframe')
validationSet = tc.SFrame('validationData.sframe')

# Train/test split by recording sessions
train, test = tc.activity_classifier.util.random_split_by_session(data, session_id='id', fraction=0.8)

# Create an activity classifier
model = tc.activity_classifier.create(train, session_id='id', prediction_window=10, target='type', validation_set=validationSet, max_iterations=100)

# Save the model for later use in Turi Create
model.save('activityClassifier.model')

# Export for use in Core ML
model.export_coreml('activityClassifier.mlmodel')

# Evaluate the model and save the results into a dictionary (might not work as data is not split evenly by classes)
metrics = model.evaluate(validationSet)
model.summary('stdout')
print(metrics['accuracy'])

Pre-processing 530 samples...

Using sequences of size 200 for model creation.

Processed a total of 53 sessions.

Iteration: 0001
	Train loss    : 0.318440728 	Train accuracy: 0.126486775
	Valid loss    : 0.099685298 	Valid accuracy: 0.267602954
Iteration: 0002
	Train loss    : 0.203466758 	Train accuracy: 0.188242104
	Valid loss    : 0.092657321 	Valid accuracy: 0.492952803
Iteration: 0003
	Train loss    : 0.184961848 	Train accuracy: 0.235860653
	Valid loss    : 0.085691366 	Valid accuracy: 0.647880850
Iteration: 0004
	Train loss    : 0.115942501 	Train accuracy: 0.471721306
	Valid loss    : 0.079006796 	Valid accuracy: 0.661965196
Iteration: 0005
	Train loss    : 0.068869354 	Train accuracy: 0.511155382
	Valid loss    : 0.072859285 	Valid accuracy: 0.746471398
Iteration: 0006
	Train loss    : 0.078281341 	Train accuracy: 0.447167978
	Valid loss    : 0.067115993 	Valid accuracy: 0.788724496
Iteration: 0007
	Train loss    : 0.050756617 	Train accuracy: 0.597464025
	Valid loss    : 0.061629525 	Valid accuracy: 0.788724496
Iteration: 0008
	Train loss    : 0.045512540 	Train accuracy: 0.652522951
	V

Iteration: 0064
	Train loss    : 0.002774139 	Train accuracy: 0.999989599
	Valid loss    : 0.015450392 	Valid accuracy: 0.957736895
Iteration: 0065
	Train loss    : 0.002860550 	Train accuracy: 0.999989599
	Valid loss    : 0.015337779 	Valid accuracy: 0.957736895
Iteration: 0066
	Train loss    : 0.002247091 	Train accuracy: 0.999989599
	Valid loss    : 0.015137821 	Valid accuracy: 0.957736895
Iteration: 0067
	Train loss    : 0.004146477 	Train accuracy: 0.976180315
	Valid loss    : 0.015000280 	Valid accuracy: 0.957736895
Iteration: 0068
	Train loss    : 0.002555191 	Train accuracy: 0.984364748
	Valid loss    : 0.014984734 	Valid accuracy: 0.957736895
Iteration: 0069
	Train loss    : 0.003382910 	Train accuracy: 0.984364748
	Valid loss    : 0.014959336 	Valid accuracy: 0.957736895
Iteration: 0070
	Train loss    : 0.003844270 	Train accuracy: 0.952371031
	Valid loss    : 0.014972508 	Valid accuracy: 0.957736895
Iteration: 0071
	Train loss    : 0.001961466 	Train accuracy: 0.976180315
	V

In [3]:
###
# Model Test
###

dat = validationSet[validationSet['id'] == 'B965DE12-BFDE-4A1C-A5F1-0468F306C7A1'] ### Neutral ###
pre = model.predict(dat, output_frequency='per_window')
print(metrics['accuracy'], pre)
model.evaluate(test)

(0.9577464788732394, Columns:
	prediction_id	int
	id	str
	class	str

Rows: 1

Data:
+---------------+-------------------------------+---------+
| prediction_id |               id              |  class  |
+---------------+-------------------------------+---------+
|       0       | B965DE12-BFDE-4A1C-A5F1-04... | neutral |
+---------------+-------------------------------+---------+
[1 rows x 3 columns]
)


{'accuracy': 0.8333333333333334,
 'auc': 0.9503472222222222,
 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 11
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 | rotateRight  |   rotateRight   |   10  |
 | hunchForward |    rotateLeft   |   10  |
 | bendBackward |   bendBackward  |   20  |
 |  rotateLeft  |    rotateLeft   |   20  |
 |  tiltRight   |    tiltRight    |   30  |
 | rotateRight  |    rotateLeft   |   10  |
 | bendForward  |   bendForward   |   10  |
 |   neutral    |     neutral     |   20  |
 |   tiltLeft   |     tiltLeft    |   20  |
 | hunchForward |   hunchForward  |   20  |
 +--------------+-----------------+-------+
 [11 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'f1_score': 0.8416666666666667,
 'log_loss': 1.5007399826025187,

type,xaccl,yaccl,zaccl,xmagnet,ymagnet,zmagnet,xgyro,ygyro,zgyro,id
